import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Finding Routes

In [322]:
import folium
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [323]:
stations_df = pd.read_csv("./saved_data/final_station_df.csv", index_col = 0)

<IPython.core.display.Javascript object>

In [324]:
non_unique_stations_df = pd.read_csv("./saved_data/non_unique_mta_stations.csv", index_col=0)

<IPython.core.display.Javascript object>

In [325]:
'A12' in non_unique_stations_df['parent_station']

False

In [326]:
non_unique_stations_df.shape

(477, 6)

In [327]:
stations_df.shape

(446, 7)

In [328]:
for x in stations_df['station_id']:
    if 'R42' in str(x):
        print(x)

R42


In [329]:
stations_df[stations_df['stop_name'] == 'Bay Ridge Av']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,station_id
471,R42,Bay Ridge Av,40.634967,-74.023377,1,NaN,R42


In [330]:
stop_times_df = pd.read_csv("./gtfs_data/stop_times.txt")

<IPython.core.display.Javascript object>

In [331]:
stop_times_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,NaN,0,0,NaN
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,NaN,0,0,NaN
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,NaN,0,0,NaN
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,NaN,0,0,NaN
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
535094,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:06:00,24:06:00,L25S,14,NaN,0,0,NaN
535095,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:07:00,24:07:00,L26S,15,NaN,0,0,NaN
535096,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:08:30,24:08:30,L27S,16,NaN,0,0,NaN
535097,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:10:30,24:10:30,L28S,17,NaN,0,0,NaN


In [332]:
for x in stop_times_df['stop_id']:
    if 'H19' in x[:-1]:
        print(x)

### Removing unnecessary columns

In [333]:
null_columns2=stop_times_df.columns[stop_times_df.isnull().any()]
stop_times_df[null_columns2].isnull().sum()

stop_headsign          535099
shape_dist_traveled    535099
dtype: int64

In [334]:
stop_times_df.drop(columns = ['stop_headsign', 'shape_dist_traveled'], inplace=True)

In [335]:
print(stop_times_df.pickup_type.nunique())
print(stop_times_df.drop_off_type.nunique())

2
2


In [336]:
# I think I need to use the stop_sequence and the stop_id to map them out 
## A lot of suspiciously round numbers
stop_times_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,0,0
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,0,0
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,0,0
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,0,0
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,0,0
...,...,...,...,...,...,...,...
535094,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:06:00,24:06:00,L25S,14,0,0
535095,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:07:00,24:07:00,L26S,15,0,0
535096,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:08:30,24:08:30,L27S,16,0,0
535097,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:10:30,24:10:30,L28S,17,0,0


## Getting Connections to Each Stop

In [337]:
# vectorizing the arrays
stop_id_order_array = np.array(stop_times_df['stop_id'])
# station_id_array = np.array(stations_df['station_id'])
# stop_id_stations_array = np.array(non_unique_stations_df['stop_id']) # will combine duplicate station_ids later
stop_id_order_array_no_ns = np.array([x[:-1] for x in stop_times_df['stop_id']]) # all stop_ids end in n or s

In [338]:
stop_id_connections_dict = {key:None for key in non_unique_stations_df['stop_id']}

In [339]:
def get_connections_to_a_stop(the_stop, stop_times_stop_id=stop_id_order_array_no_ns, df=stop_times_df):
    # make list of stop_ids that are bordering a certain stop
    # https://stackoverflow.com/questions/432112/is-there-a-numpy-function-to-return-the-first-index-of-something-in-an-array
    indices_of_the_stop = np.where(stop_times_stop_id == the_stop)
    all_connections = []
    for idx in indices_of_the_stop[0]:
        if idx > 0:
            prev_row = df.iloc[idx-1]
            if prev_row['trip_id'] == df['trip_id'][idx]:
                all_connections.append(prev_row['stop_id'][:-1])
        elif idx < df.shape[0]:
            next_row = df.iloc[idx+1]
            if next_row['trip_id'] == df['trip_id'][idx]:
                all_connections.append(next_row['stop_id'][:-1])
    connections_to_the_stop = set(all_connections)
    if the_stop in connections_to_the_stop:
        connections_to_the_stop.remove(the_stop)
    return connections_to_the_stop

In [340]:
def add_connections_to_dict(the_stop, the_dict=stop_id_connections_dict, func = get_connections_to_a_stop):
    if the_dict.get(the_stop) == None:
        connections_to_the_stop = get_connections_to_a_stop(the_stop)
        the_dict[the_stop] = connections_to_the_stop
    return the_dict

In [341]:
# took about a minute to run
for stop in non_unique_stations_df['stop_id']:
    stop_id_connections_dict = add_connections_to_dict(the_stop=stop)

In [342]:
# stop_id_connections_dict

#### Saving connections dict

In [343]:
# with open("./saved_data/all_connections_dict.pickle", "wb+") as f:
#     pickle.dump(stop_id_connections_dict, f)

### Which station has the most direct connections

In [344]:
most_connections = 0
for key, value in stop_id_connections_dict.items():
    if len(value) > most_connections:
        most_connections = len(value)
print(most_connections)

5


In [345]:
most_connections_station = ""
most_connections_station_common_name = []
for key, value in stop_id_connections_dict.items():
    if len(value) >= 5:
        most_connections_station = key
        index = non_unique_stations_df['stop_id'] == key
        common_name = list(non_unique_stations_df['stop_name'][index])
        common_name.append(f"officially known as {key}")
        most_connections_station_common_name.append(common_name)
for x in most_connections_station_common_name:
    print(x)

['Franklin Av', 'officially known as 239']
['125 St', 'officially known as 621']
['125 St', 'officially known as A15']
['59 St - Columbus Circle', 'officially known as A24']
['Hoyt - Schermerhorn Sts', 'officially known as A42']
['Kew Gardens - Union Tpke', 'officially known as F06']
['Jackson Hts - Roosevelt Av', 'officially known as G14']
['Myrtle Av', 'officially known as M11']
['DeKalb Av', 'officially known as R30']
['Atlantic Av - Barclays Ctr', 'officially known as R31']
['36 St', 'officially known as R36']


### Putting Connections into a dataframe

#### Get All Combinations of Stops

In [346]:
# only need one combination of each pair (not each permutation)
## e.g.: 101 <-> 103 should have same amount of time between them
permutation_list = []
for key, value in stop_id_connections_dict.items():
    for item in value:
        permutation = [key, item]
        if [item, key] not in permutation_list:
            permutation_list.append(permutation)

In [347]:
# there are 551 distinct station to station pairs in the NYC subway system
len(permutation_list)

551

In [348]:
# permutation_list

#### Making sure second avenue subway cars are in list

In [349]:
# new 96 st station has the id 625 which is in the dict! 
# for idx, x in enumerate(non_unique_stations_df['stop_name']):
#     if '96' in x:
#         print(non_unique_stations_df.iloc[idx])

#### Putting combos in df

In [350]:
start_station = [x[0] for x in permutation_list]
end_station = [x[1] for x in permutation_list]

In [351]:
edge_list_df = pd.DataFrame(data=[start_station, end_station]).T

<IPython.core.display.Javascript object>

In [352]:
edge_list_df.columns = ['node1', 'node2']
edge_list_df['trail'] = 'nyc subway' # making this a single trail for simplicity's sake
edge_list_df['color'] = 'red'

## Getting Distance Between Each Connection

In [353]:
# will add this connection "distance" to the distance column in the edge_list_df
## first: are the time columns in stop_times_df timestamps? can I subtract them?
stop_times_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,0,0
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,0,0
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,0,0
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,0,0
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,0,0


In [354]:
type(stop_times_df['arrival_time'][0])

str

In [355]:
neg_times = []
over_24 = []
for x in stop_times_df['arrival_time']:
    if '-' in x:
        neg_times.append(x)
    elif '24' in x:
        over_24.append(x)

In [356]:
len(neg_times)

0

In [357]:
len(over_24)

20019

In [358]:
arrival_times_datetime = pd.to_datetime(stop_times_df['arrival_time'], errors = 'coerce', infer_datetime_format=True)

<IPython.core.display.Javascript object>

In [359]:
type(arrival_times_datetime[0])

pandas._libs.tslibs.timestamps.Timestamp

In [360]:
arrival_times_datetime[0]

Timestamp('2020-07-26 00:06:00')

In [361]:
arrival_times_timedelta = pd.to_timedelta(arrival_times_datetime, unit = 'seconds', errors = 'coerce')

<IPython.core.display.Javascript object>

In [362]:
arrival_times_timedelta.isnull().sum()

18044

In [363]:
stop_times_df['arrival_times_timedelta'] = arrival_times_timedelta
edge_list_df['distance'] = 0 # empty column where I will add the distance between nodes

### Checking if there is a difference in time between stops depending on day of week

In [364]:
indices_of_the_stop = np.where(stop_id_order_array_no_ns == non_unique_stations_df['stop_id'][0])

In [365]:
sat = stop_times_df[stop_times_df.trip_id.str.contains("AFA19GEN-1038-Saturday")]
sun = stop_times_df[stop_times_df.trip_id.str.contains("AFA19GEN-1037-Sunday")]
weekday = stop_times_df[stop_times_df.trip_id.str.contains("AFA19GEN-1087-Weekday")]

In [366]:
# looks like it's the same 1:30 each way at all times
sun_and_first_2_stops = sun[(sun.stop_id.str.contains("101") | (sun.stop_id.str.contains("103")))]
sat_and_first_2_stops = sat[(sat.stop_id.str.contains("101") | (sat.stop_id.str.contains("103")))]
weekday_and_first_2_stops = weekday[(weekday.stop_id.str.contains("101") | (weekday.stop_id.str.contains("103")))]

### Finding "distance" between each node

In [367]:
edge_list_df

,node1,node2,trail,color,distance
0,101,103,nyc subway,red,0
1,103,104,nyc subway,red,0
2,104,106,nyc subway,red,0
3,106,107,nyc subway,red,0
4,107,108,nyc subway,red,0
...,...,...,...,...,...
546,R42,R43,nyc subway,red,0
547,R43,R44,nyc subway,red,0
548,R44,R45,nyc subway,red,0
549,S01,S03,nyc subway,red,0


- choose node
- look for node
- look for second node (immediately following)
- compute timedelta between them
- add timedelta to correct row
- move onto next row in df

In [368]:
def find_edge_distance(row_of_edge_list_df, df=edge_list_df, stop_times_df=stop_times_df):
    null_list = stop_times_df['arrival_times_timedelta'].isnull()
    node_location = row_of_edge_list_df
    if (df['distance'][node_location] == 0):
        node1 = df['node1'][node_location]
        node2 = df['node2'][node_location]
        indices_of_the_stop = np.where(stop_id_order_array_no_ns == node1)
        time_delta = 0
        for idx in indices_of_the_stop[0]:
            next_row = stop_times_df.iloc[idx+1]
            stop_id_dest = next_row['stop_id'][:-1]
            if stop_id_dest == node2:
                if (null_list[idx+1] == False) & (null_list[idx] == False):
                    if stop_times_df['trip_id'][idx+1] == stop_times_df['trip_id'][idx]:
                        time_delta = stop_times_df['arrival_times_timedelta'][idx+1] - stop_times_df['arrival_times_timedelta'][idx]
                        df['distance'][node_location] = time_delta.seconds
                        break
    return df

In [369]:
for x in range(0, len(edge_list_df)):
    edge_list_df = find_edge_distance(x)

In [370]:
# no nulls!
edge_list_df['distance'].isnull().sum()

0

In [371]:
edge_list_df['distance'].min()

0

In [372]:
# fix the duplicate stations (at least in node1 and node2 lists)
## might have to make all nodes with only one connection required

### Fixing the duplicate stations

In [373]:
duplicate_stations = [x for x in stations_df['station_id'] if len(x) > 4]

In [374]:
duplicate_stations

['112_A09',
 '125_A24',
 '222_415',
 '140_142',
 '414_D11',
 '232_423',
 '235_D24_R31',
 '710_G14',
 '631_723_901',
 '635_L03_R20',
 'D17_R17',
 '718_R09',
 '719_F09_G22',
 '127_725_902_R16',
 'A12_D13',
 'L01_A31',
 'A32_D20',
 'A38_M22',
 'A41_R29',
 'A45_S01',
 'D43_N12',
 'M12_G31',
 'H01_H02',
 'H04_H19',
 'L22_J27',
 'L17_M08',
 'Q01_R23']

In [375]:
duplicate_stations_ind1 = [x.split("_") for x in duplicate_stations]
duplicate_stations_ind = [val for sublist in duplicate_stations_ind1 for val in sublist]
# list(np.array(duplicate_stations_ind1).flat)

In [376]:
# duplicate_stations_ind

In [377]:
stations_w_station_id = {key:None for key in duplicate_stations_ind}

In [378]:
counter = 0
keys_list = list(stations_w_station_id.keys())
for item in keys_list:
    for thing in duplicate_stations:
        if item in thing:
            stations_w_station_id[item] = thing

In [379]:
# stations_w_station_id

#### Adding south ferry and broad channel 

In [380]:
stations_w_station_id['140'] = '140_142'
stations_w_station_id['142'] = '140_142'
stations_w_station_id['H04'] = 'H04_H19'
stations_w_station_id['H19'] = 'H04_H19'
stations_w_station_id['A13'] = 'A12_D13'
stations_w_station_id['A13_D13'] = 'A12_D13'

In [381]:
edge_list_df.replace(to_replace=stations_w_station_id, inplace=True)

#### Fixing Cases that Slipped through the cracks

In [382]:
# "R45" should really be R_42
# http://web.mta.info/developers/data/nyct/subway/Stations.csv
# I accidentally wrote "A13" when I meant "A12" in nodelist 

In [383]:
edge_list_df[edge_list_df['node2'] == 'R45']

,node1,node2,trail,color,distance
548,R44,R45,nyc subway,red,300


In [384]:
edge_list_df['node2'][548] = 'R42'

#### Checking that there are no duplicate nodes 

In [385]:
multi_station_stop_indices = []
for idx, x in enumerate(edge_list_df['node2']):
    if len(x) > 4:
        multi_station_stop_indices.append(idx)

In [386]:
# edge_list_df.iloc[multi_station_stop_indices]

### Fixing stations with errors in distance

In [387]:
# is actually legit b/c an express train makes this route
edge_list_df.distance.max()

780

In [388]:
edge_list_df.distance.sort_values()

409      0
110      0
49       0
263      0
408      0
      ... 
375    570
90     630
503    660
314    660
56     780
Name: distance, Length: 551, dtype: int64

In [389]:
edge_list_df.iloc[45:50]

,node1,node2,trail,color,distance
45,205,206,nyc subway,red,60
46,206,207,nyc subway,red,60
47,207,208,nyc subway,red,90
48,208,209,nyc subway,red,90
49,208,213,nyc subway,red,0


In [390]:
# 48, 110 looks like a mistake
# 408 share the same parent station
# 264, 409 is b/c of a northbound v southbound issue
edge_list_df[edge_list_df['distance'] == 0]

,node1,node2,trail,color,distance
49,208,213,nyc subway,red,0
110,409,413,nyc subway,red,0
263,A61,H01_H02,nyc subway,red,0
408,H01_H02,H01_H02,nyc subway,red,0
409,H01_H02,A61,nyc subway,red,0


#### Dropping mistaken connections

In [391]:
# duplicate rows after combining stations
# some repeat routes, will pick lower value if values are different 
multi_station_edgelist = []
for x in range(len(edge_list_df)):
    if len(edge_list_df['node1'][x]) > 4 or len(edge_list_df['node2'][x]) > 4:
        multi_station_edgelist.append(x)
edge_list_df.iloc[multi_station_edgelist]

,node1,node2,trail,color,distance
8,111,112_A09,nyc subway,red,120
9,112_A09,113,nyc subway,red,90
23,123,127_725_902_R16,nyc subway,red,240
24,124,125_A24,nyc subway,red,90
25,125_A24,126,nyc subway,red,120
...,...,...,...,...,...
533,A41_R29,R30,nyc subway,red,90
534,R30,235_D24_R31,nyc subway,red,180
535,235_D24_R31,R32,nyc subway,red,120
536,235_D24_R31,R36,nyc subway,red,390


In [392]:
node1_items = set(edge_list_df['node1'][multi_station_edgelist])
node2_items = set(edge_list_df['node2'][multi_station_edgelist])

In [393]:
multi_station_edgelist

[8,
 9,
 23,
 24,
 25,
 26,
 27,
 42,
 64,
 65,
 66,
 74,
 75,
 77,
 78,
 79,
 80,
 114,
 115,
 116,
 117,
 122,
 158,
 159,
 160,
 161,
 164,
 165,
 166,
 179,
 180,
 182,
 187,
 188,
 189,
 191,
 192,
 193,
 194,
 195,
 200,
 201,
 202,
 204,
 205,
 206,
 208,
 209,
 210,
 218,
 219,
 220,
 221,
 226,
 227,
 228,
 229,
 230,
 234,
 235,
 236,
 237,
 238,
 239,
 245,
 246,
 263,
 267,
 286,
 293,
 297,
 298,
 299,
 304,
 305,
 306,
 308,
 309,
 314,
 315,
 316,
 338,
 340,
 353,
 354,
 383,
 388,
 389,
 390,
 397,
 402,
 403,
 408,
 409,
 410,
 411,
 412,
 413,
 441,
 442,
 443,
 451,
 452,
 453,
 463,
 464,
 467,
 468,
 477,
 478,
 481,
 483,
 490,
 491,
 501,
 502,
 503,
 504,
 512,
 513,
 517,
 518,
 519,
 520,
 521,
 523,
 524,
 526,
 527,
 532,
 533,
 534,
 535,
 536,
 549]

In [394]:
repeat_list = []
for item in node1_items:
    if len(item) < 4:
        for thing in node2_items:
            if item == thing:
                repeat_list.append(item)

In [395]:
repeat_list

['L02', 'H03', 'A15', 'A14', 'A61', '126', 'D12', 'R30', '724', 'R22', 'A40']

In [396]:
edge_list_df.iloc[408]

node1          H01_H02
node2          H01_H02
trail       nyc subway
color              red
distance             0
Name: 408, dtype: object

In [397]:
# dropping rows with same node in both cols
for x in range(len(edge_list_df)):
    if edge_list_df['node1'][x] == edge_list_df['node2'][x]:
        edge_list_df.drop(index=[x], inplace=True)

In [398]:
non_unique_stations_df[non_unique_stations_df.stop_id == '208']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
132,208,Gun Hill Rd,40.87785,-73.866256,1,NaN


In [399]:
non_unique_stations_df[non_unique_stations_df.stop_id == '213']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
147,213,E 180 St,40.841894,-73.873488,1,NaN


In [400]:
edge_list_df.reset_index(drop=True, inplace=True)
for x in range(len(edge_list_df)):
    if edge_list_df['node1'][x] == '409' and edge_list_df['node2'][x] == '413':
        edge_list_df.drop(index=[x], inplace=True)
    elif edge_list_df['node1'][x] == '208' and edge_list_df['node2'][x] == '213':
        edge_list_df.drop(index=[x], inplace=True)

In [401]:
# edge_list_df.drop(index=[408, 48, 110], inplace=True) Forget y i wanted to drop these but i put it in conditions above
# edge_list_df.drop(index=[404, 204, 207], inplace=True)

#### Manually finding values for the ones still at 0 

- 4 minutes between Aqueduct - N Conduit Av and Rockaway Blvd

In [403]:
# fixing the 2 zeroes
edge_list_df.reset_index(drop=True, inplace=True)
for idx, x in enumerate(edge_list_df['distance']):
    if edge_list_df['node1'][idx] == 'H01_H02' and edge_list_df['node2'][idx] == 'A61':
        edge_list_df.distance[idx] = 4 * 60
    elif edge_list_df['node2'][idx] == 'H01_H02' and edge_list_df['node1'][idx] == 'A61':
        edge_list_df.distance[idx] = 4 * 60

In [404]:
# edge_list_df['distance'][264] = 4 * 60
# edge_list_df['distance'][409] = 4 * 60

## Saving edge_list_df

In [405]:
edge_list_df.reset_index(drop=True, inplace=True)

In [407]:
# edge_list_df.distance.sort_values()

### Checking for duplicate entries

In [411]:
duplicate_indices_to_drop = []
edge_list_df.reset_index(drop=True, inplace=True)
for x in range(len(edge_list_df)):
    for i in range(x+1, len(edge_list_df)):
        if edge_list_df['node1'][x] == edge_list_df['node1'][i] and \
        edge_list_df['node2'][x] == edge_list_df['node2'][i]:
            dist1 = edge_list_df['distance'][x]
            dist2 = edge_list_df['distance'][i]
            if dist1 < dist2:
                duplicate_indices_to_drop.append(i)
            elif dist1 > dist2:
                duplicate_indices_to_drop.append(x)
        elif edge_list_df['node2'][x] == edge_list_df['node1'][i] and \
        edge_list_df['node1'][x] == edge_list_df['node2'][i]:
            dist1 = edge_list_df['distance'][x]
            dist2 = edge_list_df['distance'][i]
            if dist1 < dist2:
                duplicate_indices_to_drop.append(i)
            elif dist1 > dist2:
                duplicate_indices_to_drop.append(x)

In [412]:
edge_list_df.drop(index = duplicate_indices_to_drop, inplace=True)

In [413]:
edge_list_df.reset_index(inplace=True, drop=True)

In [416]:
edge_list_df.to_csv("./saved_data/edge_list_df_no_req.csv")

In [415]:
edge_list_df.shape

(544, 5)